<a href="https://colab.research.google.com/github/abrodyyy/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 https://cloud.r-project.org/bin/linu

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-02-06 03:25:03--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.33MB/s    in 0.2s    

2023-02-06 03:25:03 (6.33 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Pet_Products_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

### Create DataFrames to match tables

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| REAKC26P07MDN|          5|            0|          0|   N|                Y|
|R3NU7OMZ4HQIEG|          2|            0|          1|   N|                Y|
|R14QJW3XF8QO1P|          5|            0|          0|   N|                Y|
|R2HB7AX0394ZGY|          5|            0|          0|   N|                Y|
| RGKMPDQGSAHR3|          5|            0|          0|   N|                Y|
|R1DJCVPQGCV66E|          5|            0|          0|   N|                Y|
|R3V52EAWLPBFQG|          3|            0|          0|   N|                Y|
|R3DKO8J1J28QBI|          2|            0|          0|   N|                Y|
| R764DBXGRNECG|          5|            1|          1|   N|                N|
| RW1853GAT0Z9F|          5|            0|          0|   N|     

In [ ]:
# Filter the data and create a new DataFrame for rows where total_votes >= 20
votes_df = vine_df.filter("total_votes >= 20")
votes_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|R3A1VQ3SQDXEJ3|          5|           20|         20|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|     

In [ ]:
# Filter votes_df and create a new DataFrame for rows where helpful_votes >= 50%
helpful_votes_df = vine_df.filter("helpful_votes/total_votes >= 0.5")
helpful_votes_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R764DBXGRNECG|          5|            1|          1|   N|                N|
|R1H7AVM81TAYRV|          1|            2|          2|   N|                Y|
| RMB8N0DBRH34O|          5|            1|          1|   N|                Y|
| RLR047V0C09SG|          5|            1|          1|   N|                Y|
|R2D06CYY6KZSJ0|          2|           16|         19|   N|                Y|
|R2P2G1XUTS2UI1|          5|            8|          8|   N|                Y|
|R2E9I9L1DXK7U8|          5|           10|         10|   N|                Y|
|  ROPI31U4NHTB|          2|            1|          1|   N|                Y|
|R2EDA9TQWM5LOW|          5|            1|          1|   N|                Y|
| RBJ69AC0G5A5R|          5|            1|          1|   N|     

In [ ]:
# Filter helpful_votes_df and create a new DataFrame for rows where a review was
# written as part of the Vine program (paid)
vine_reviews_paid_df = helpful_votes_df.filter(helpful_votes_df ["vine"] == "Y")
vine_reviews_paid_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R9WD02GLN4A35|          5|            7|          8|   Y|                N|
|R10A8UNLH0KARF|          3|            1|          1|   Y|                N|
|R3M4YYUXIJ6XMF|          4|            2|          2|   Y|                N|
|R22RZB3DND8XJO|          4|            1|          1|   Y|                N|
| R6PROVFEGSNRK|          3|            1|          1|   Y|                N|
|R1X65DWI7Y9TKP|          4|            1|          2|   Y|                N|
| R18GPJ1M4VRPH|          2|            2|          2|   Y|                N|
|R3A71VR1JZD8WF|          2|           27|         30|   Y|                N|
| RYVR28GFFVAVB|          5|            5|          6|   Y|                N|
| RA9DDSJW1ST84|          4|            6|          6|   Y|     

In [ ]:
# Filter helpful_votes_df and create a new DataFrame for rows where a review was
# NOT written as part of the Vine program (unpaid)
vine_reviews_unpaid_df = helpful_votes_df.filter(helpful_votes_df ["vine"] == "N")
vine_reviews_unpaid_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R764DBXGRNECG|          5|            1|          1|   N|                N|
|R1H7AVM81TAYRV|          1|            2|          2|   N|                Y|
| RMB8N0DBRH34O|          5|            1|          1|   N|                Y|
| RLR047V0C09SG|          5|            1|          1|   N|                Y|
|R2D06CYY6KZSJ0|          2|           16|         19|   N|                Y|
|R2P2G1XUTS2UI1|          5|            8|          8|   N|                Y|
|R2E9I9L1DXK7U8|          5|           10|         10|   N|                Y|
|  ROPI31U4NHTB|          2|            1|          1|   N|                Y|
|R2EDA9TQWM5LOW|          5|            1|          1|   N|                Y|
| RBJ69AC0G5A5R|          5|            1|          1|   N|     

In [ ]:
# Determine the total number of reviews, the number of 5-star reviews, and the 
# percentage of 5-star reviews for the two types of review (paid vs unpaid).
from pyspark.sql.functions import count, col, when

vine_reviews_all_df = helpful_votes_df.groupBy("vine").agg(
    count(col("vine")).alias("Total_Reviews"),
    count(when(col("star_rating") == 5, True)).alias("5_Star_Reviews"),
    (count(when(col("star_rating") == 5, True)) / count(col("vine")) * 100).alias("5_Star_Reviews_Percentage")).show()

+----+-------------+--------------+-------------------------+
|vine|Total_Reviews|5_Star_Reviews|5_Star_Reviews_Percentage|
+----+-------------+--------------+-------------------------+
|   Y|         4103|          1704|         41.5305873750914|
|   N|       816463|        468620|       57.396354764392264|
+----+-------------+--------------+-------------------------+

